In [6]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import torch.optim
from agent.agents import WaterworldAgent, DisjointWaterWorldAgent
from agent.evolve import EvoController
import numpy as np
import pickle

In [7]:
agent = DisjointWaterWorldAgent(num_nodes=4, spatial=7, channels=3, input_channels=2)
print(agent.input_channels)
obs_data = np.loadtxt("/Users/loggiasr/Projects/ReIntAI/models/pretrain_data/obs_data.txt")
reward_data = np.loadtxt("/Users/loggiasr/Projects/ReIntAI/models/pretrain_data/returns.txt")
print(obs_data.shape)
print(reward_data.shape)

In [3]:
from sklearn.linear_model import LinearRegression
glm = LinearRegression(fit_intercept=False)
glm.fit(obs_data, reward_data)
# res = glm.predict(obs_data)
# print(reward_data - res)
agent.value_decoder = torch.nn.Parameter(torch.from_numpy(glm.coef_).float().reshape(-1, 1))
# agent.value_decoder.bias = torch.nn.Parameter(torch.from_numpy(np.array([glm.intercept_])).float().flatten())

In [8]:
agent.pretrain_agent_input(5000, obs_data)

In [6]:
fig, axs = plt.subplots(2)
axs[0].plot(loss)
axs[1].plot(val_loss)
plt.show()

In [9]:
with open("/Users/loggiasr/Projects/ReIntAI/models/wworld_pretrain7.pkl", "wb") as f:
    pickle.dump(agent.input_encoder, f)
with open("/Users/loggiasr/Projects/ReIntAI/models/wworld_val_pretrain7.pkl", "wb") as f:
    pickle.dump(agent.value_decoder, f)

In [2]:
agent = WaterworldAgent(num_nodes=4, spatial=7, channels=3, input_channels=3)
with open("/Users/loggiasr/Projects/ReIntAI/models/wworld_pretrain7.pkl", "rb") as f:
    in_enc = pickle.load(f)
agent.input_encoder = in_enc

In [11]:
with open("/Users/loggiasr/Projects/ReIntAI/models/wworld_1.pkl", "rb") as f:
    evo = pickle.load(f)

In [3]:
evo = EvoController(seed_agent=agent, epochs=5, num_base=5)

In [ ]:
evo.controller(mp=False)

In [9]:
evo.plot_loss()

In [7]:
with open("/Users/loggiasr/Projects/ReIntAI/models/wworld_pt_e200.pkl", "wb") as f:
    pickle.dump(evo, f)

In [15]:
evo.instances = 1
evo.num_agents = 1
evo.base_agent[0].input_channels = 3
evo.base_agent[0].core_model.detach(reset_intrinsic=True)
evo.play(human_interface=True, cyles=500)